# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 6 2023 9:03AM,256677,10,0086355717,ISDIN Corporation,Released
1,Feb 6 2023 9:03AM,256677,10,0086356425,ISDIN Corporation,Released
2,Feb 6 2023 9:02AM,256675,10,0086356427,ISDIN Corporation,Released
3,Feb 6 2023 9:02AM,256675,10,0086356434,ISDIN Corporation,Released
4,Feb 6 2023 9:02AM,256675,10,0086356408,ISDIN Corporation,Released
5,Feb 6 2023 9:02AM,256675,10,0086356426,ISDIN Corporation,Released
6,Feb 6 2023 9:02AM,256675,10,0086356373,ISDIN Corporation,Released
7,Feb 6 2023 9:02AM,256675,10,0086356376,ISDIN Corporation,Released
8,Feb 6 2023 9:02AM,256675,10,0086356378,ISDIN Corporation,Released
9,Feb 6 2023 9:02AM,256675,10,0086356374,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,256673,Released,36
17,256674,Released,52
18,256675,Released,31
19,256676,Released,2
20,256677,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
256673,NaN,36.0
256674,NaN,52.0
256675,NaN,31.0
256676,NaN,2.0
256677,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256651,1.0,0.0
256655,1.0,1.0
256656,0.0,3.0
256659,1.0,0.0
256661,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256651,1,0
256655,1,1
256656,0,3
256659,1,0
256661,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256651,1,0
1,256655,1,1
2,256656,0,3
3,256659,1,0
4,256661,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256651,1,
1,256655,1,1
2,256656,,3
3,256659,1,
4,256661,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 6 2023 9:03AM,256677,10,ISDIN Corporation
2,Feb 6 2023 9:02AM,256675,10,ISDIN Corporation
33,Feb 6 2023 9:02AM,256674,10,ISDIN Corporation
85,Feb 6 2023 9:02AM,256673,10,ISDIN Corporation
121,Feb 6 2023 9:02AM,256671,10,ISDIN Corporation
169,Feb 6 2023 9:02AM,256670,10,ISDIN Corporation
204,Feb 6 2023 9:02AM,256669,10,ISDIN Corporation
248,Feb 6 2023 8:57AM,256676,19,VITABIOTICS USA INC.
250,Feb 6 2023 8:50AM,256668,10,ISDIN Corporation
257,Feb 6 2023 8:46AM,256667,12,"Uniderm USA, Inc"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 6 2023 9:03AM,256677,10,ISDIN Corporation,,2
1,Feb 6 2023 9:02AM,256675,10,ISDIN Corporation,,31
2,Feb 6 2023 9:02AM,256674,10,ISDIN Corporation,,52
3,Feb 6 2023 9:02AM,256673,10,ISDIN Corporation,,36
4,Feb 6 2023 9:02AM,256671,10,ISDIN Corporation,,48
5,Feb 6 2023 9:02AM,256670,10,ISDIN Corporation,,35
6,Feb 6 2023 9:02AM,256669,10,ISDIN Corporation,,44
7,Feb 6 2023 8:57AM,256676,19,VITABIOTICS USA INC.,,2
8,Feb 6 2023 8:50AM,256668,10,ISDIN Corporation,,7
9,Feb 6 2023 8:46AM,256667,12,"Uniderm USA, Inc",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 9:03AM,256677,10,ISDIN Corporation,2,
1,Feb 6 2023 9:02AM,256675,10,ISDIN Corporation,31,
2,Feb 6 2023 9:02AM,256674,10,ISDIN Corporation,52,
3,Feb 6 2023 9:02AM,256673,10,ISDIN Corporation,36,
4,Feb 6 2023 9:02AM,256671,10,ISDIN Corporation,48,
5,Feb 6 2023 9:02AM,256670,10,ISDIN Corporation,35,
6,Feb 6 2023 9:02AM,256669,10,ISDIN Corporation,44,
7,Feb 6 2023 8:57AM,256676,19,VITABIOTICS USA INC.,2,
8,Feb 6 2023 8:50AM,256668,10,ISDIN Corporation,7,
9,Feb 6 2023 8:46AM,256667,12,"Uniderm USA, Inc",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 9:03AM,256677,10,ISDIN Corporation,2,
1,Feb 6 2023 9:02AM,256675,10,ISDIN Corporation,31,
2,Feb 6 2023 9:02AM,256674,10,ISDIN Corporation,52,
3,Feb 6 2023 9:02AM,256673,10,ISDIN Corporation,36,
4,Feb 6 2023 9:02AM,256671,10,ISDIN Corporation,48,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 9:03AM,256677,10,ISDIN Corporation,2.0,NaN
1,Feb 6 2023 9:02AM,256675,10,ISDIN Corporation,31.0,NaN
2,Feb 6 2023 9:02AM,256674,10,ISDIN Corporation,52.0,NaN
3,Feb 6 2023 9:02AM,256673,10,ISDIN Corporation,36.0,NaN
4,Feb 6 2023 9:02AM,256671,10,ISDIN Corporation,48.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 6 2023 9:03AM,256677,10,ISDIN Corporation,2.0,0.0
1,Feb 6 2023 9:02AM,256675,10,ISDIN Corporation,31.0,0.0
2,Feb 6 2023 9:02AM,256674,10,ISDIN Corporation,52.0,0.0
3,Feb 6 2023 9:02AM,256673,10,ISDIN Corporation,36.0,0.0
4,Feb 6 2023 9:02AM,256671,10,ISDIN Corporation,48.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2566708,257.0,0.0
12,769994,59.0,0.0
16,513323,1.0,1.0
19,1026646,6.0,2.0
22,256651,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2566708,257.0,0.0
1,12,769994,59.0,0.0
2,16,513323,1.0,1.0
3,19,1026646,6.0,2.0
4,22,256651,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,257.0,0.0
1,12,59.0,0.0
2,16,1.0,1.0
3,19,6.0,2.0
4,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,257.0
1,12,Released,59.0
2,16,Released,1.0
3,19,Released,6.0
4,22,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,22
Status,,,,,
Executing,0.0,0.0,1.0,2.0,1.0
Released,257.0,59.0,1.0,6.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,22
0,Executing,0.0,0.0,1.0,2.0,1.0
1,Released,257.0,59.0,1.0,6.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,22
0,Executing,0.0,0.0,1.0,2.0,1.0
1,Released,257.0,59.0,1.0,6.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()